In [1]:
import numpy as np
from PIL import Image
from skimage.transform import rescale
from skimage import measure as sk_measure
import itertools
import math
import time
from datetime import datetime as dt
import collections
import re

def get_input(day, split=None, f=str.strip):
    fin = f'dat/2015-{day:02}.txt'
    if split is None:
        input = open(fin, 'r').readlines()
    else:
        input = open(fin, 'r').read().split(split)
    
    if f is not None:
        input = [f(x) for x in input]
    
    return input

def timestamp(ts):
    date = dt.fromtimestamp(ts)
    return f'[{date.hour:02}:{date.minute:02}:{date.second:02}]'

def output(*args, ts=0):
    out = ' '.join([str(x) for x in args])
    if ts:
        ts = timestamp(ts)
        sz = 76 - len(out) - len(ts)
        if sz > 0:
            out += ' ' * sz
        out += ts
    print(out)

def as_ints(input):    
    nums = []
    for x in input:
        x = [int(c) for c in x]
        nums.append(np.asarray(x))
    return np.asarray(nums)

# Day 1

In [3]:
input = get_input(1)

def day_one(a, two=False):
    floor = 0
    idx = 0
    for row in a:
        for c in row:
            idx += 1
            floor += 1 if c == '(' else -1
            if two and floor < 0:
                return idx
    return floor
        
    

output(day_one(input), ts=1733259903)
output(day_one(input, True), ts=1733260045)

280                                                               [16:05:03]
1797                                                              [16:07:25]


# Day 2

In [10]:
input = get_input(2)
        
def day_two(a, two=False):
    total = 0
    for row in a:
        l, w, h = [int(i) for i in row.split('x')]
        if two:
            lst = [l, w, h]
            x = lst.pop(lst.index(min(lst)))
            y = lst.pop(lst.index(min(lst)))
            total += 2*x + 2*y + l*w*h
        else:
            total += 2*l*w + 2*w*h + 2*h*l + min(l*w, w*h, h*l)
    return total
        
output(day_two(input), ts=1733261074)
output(day_two(input, True), ts=1733263415)

1586300                                                           [16:24:34]
3737498                                                           [17:03:35]


# Day 3

In [28]:
input = get_input(3)

xmap = {
    '<': -1,
    '>': 1
}
ymap = {
    '^': -1,
    'v': 1
}

def day_three(a, two=False):
    a = a[0]
    grid = np.zeros((len(a)*2+1, len(a)*2+1), np.int32)
    y = x = len(a)
    grid[y][x] += 1
    
    if two:
        y1 = y
        x1 = x
        grid[y1][x1] += 1
        
    cnt = 0
    for c in a:
        cnt += 1
        if two and cnt % 2 == 0:
            x1 += xmap.get(c, 0)
            y1 += ymap.get(c, 0)
            grid[y1][x1] += 1
        else:
            x += xmap.get(c, 0)
            y += ymap.get(c, 0)
            grid[y][x] += 1
            
    grid[np.where(grid>0)] = 1
    return np.sum(grid)

output(day_three(input), ts=1733266367) #34873487 low
output(day_three(input, True), ts=1733267751)

2592                                                              [17:52:47]
2360                                                              [18:15:51]


# Day 4

In [47]:
input = "iwrupvqb"
#input = "abcdef"
import hashlib

def day_four(a, two=False):
    i = 0
    while True:
        i += 1
        h = hashlib.md5(f"{a}{i}".encode()).hexdigest()
        if two and h[:6] == "000000":
            return i
        elif not two and h[:5] == "00000":
            return i
        
output(day_four(input), ts=1733290413)
output(day_four(input, True), ts=1733290716)

346386                                                            [00:33:33]
9958218                                                           [00:38:36]


# Day 5

In [28]:
input = """ugknbfddgicrmopn
aaa
jchzalrnumimnmhp
haegwjzuvuyypxyu
dvszwmarrgswjxmb""".split('\n')
input = """qjhvhtzxzqqjkmpb
xxyxx
uurcxstgmygtbstg
ieodomkazucvgmuy""".split('\n')
input = get_input(5)

def day_five(a, two=False):
    nice = 0
    for row in a:
        naughty = False
        if two:
            for i in range(len(row)-2):
                c = row[i] + row[i+1]
                if row[i+2:].find(c) >= 0:
                    break
            else:
                naughty = True
            
            for i in range(len(row)-2):
                if row[i] == row[i+2]:
                    break
            else:
                naughty = True
        else:
            for s in ('ab', 'cd', 'pq', 'xy'):
                if s in row:
                    naughty = True
                    break

            cnt = 0
            for v in "aeiou":
                cnt += row.count(v)
            if cnt < 3:
                naughty = True

            last = ""
            for c in row:
                if c == last:
                    break
                last = c
            else:
                naughty = True
        
        if not naughty:
            nice += 1
    
    return nice
    
    

output(day_five(input), ts=1733375557)
output(day_five(input, True), ts=1733376076) #48, 41, 43

236                                                               [00:12:37]
51                                                                [00:21:16]


# Day 6

In [58]:
input = get_input(6)

def day_six(a, two=False):
    if two:
        grid = np.zeros((1000,1000), np.int32)
    else:
        grid = np.zeros((1000,1000), bool)
    
    for row in a:
        inst = row.split()
        if len(inst) == 4:
            ul = inst[1]
            lr = inst[3]
            inst = inst[0]
        else:
            ul = inst[2]
            lr = inst[4]
            inst = inst[1]
        
        x0, y0 = [int(i) for i in ul.split(',')]
        x1, y1 = [int(i) for i in lr.split(',')]
        
        if inst == 'on':
            if two:
                grid[y0:y1+1,x0:x1+1] += 1
            else:
                grid[y0:y1+1,x0:x1+1] = 1
        elif inst == 'off':
            if two:
                grid[y0:y1+1,x0:x1+1] -= 1
                grid[np.where(grid<0)] = 0
            else:
                grid[y0:y1+1,x0:x1+1] = 0
        else:
            if two:
                grid[y0:y1+1,x0:x1+1] += 2
            else:
                grid[y0:y1+1,x0:x1+1] = np.invert(grid[y0:y1+1,x0:x1+1])
        
    return np.sum(grid)
        
output(day_six(input), ts=1638334897) # 496615 hi
output(day_six(input, True), ts=1638335082)

377891                                                            [00:01:37]
14110788                                                          [00:04:42]


# Day 7

In [95]:
input = """123 -> x
456 -> y
x AND y -> d
x OR y -> e
x LSHIFT 2 -> f
y RSHIFT 2 -> g
NOT x -> h
NOT y -> i""".split('\n')
input = get_input(7)

def run_circuit(a, circuit):
    done = False
    a.sort()
    while not done:
        #print(circuit)
        done = True
        for row in a:
            row = row.split()
            dst = row[-1]
            if dst in circuit:
                continue
                
            if 'NOT' in row:
                src = row[1]
                if src not in circuit:
                    done = False
                    continue
                circuit[dst] = ~circuit[src]
            elif 'RSHIFT' in row:
                src = row[0]
                val = int(row[2])
                if src not in circuit:
                    done = False
                    continue
                circuit[dst] = circuit[src] >> val
            elif 'LSHIFT' in row:
                src = row[0]
                val = int(row[2])
                if src not in circuit:
                    done = False
                    continue
                circuit[dst] = circuit[src] << val
            elif 'OR' in row:
                src1 = row[0]
                src2 = row[2]
                if src1 not in circuit or src2 not in circuit:
                    done = False
                    continue
                circuit[dst] = circuit[src1] | circuit[src2]
            elif 'AND' in row:
                src1 = row[0]
                if src1 == '1':
                    src1 = 1
                elif src1 in circuit:
                    src1 = circuit[src1]
                else:
                    done = False
                    continue
                    
                src2 = row[2]                
                if src2 not in circuit:
                    done = False
                    continue
                circuit[dst] = src1 & circuit[src2]
            else:
                src = row[0]
                try:
                    val = int(src)
                    circuit[dst] = np.uint16(val)
                except ValueError:
                    if src not in circuit:
                        done = False
                        continue
                    circuit[dst] = circuit[src]

    return circuit

def day_seven(a, two=False):
    circuit = run_circuit(a, {})
    if two:
        circuit = run_circuit(a, {'b':circuit['a']})
    return circuit['a']
        

output(day_seven(input), ts=1638334897)
output(day_seven(input, True), ts=1638335082)

956                                                               [00:01:37]
40149                                                             [00:04:42]


# Day 8

In [144]:
input = """
""
"abc"
"aaa\\"aaa"
"\\x27"
""".split('\n')
input = get_input(8)

def day_eight(a, two=False):
    total = 0
    for row in a:
        if not row:continue
        if two:
            sub = row.replace('\\\\', '\\\\\\\\').replace('\\x', '\\\\x').replace('\\"', '\\\\\\"')
            if sub.endswith('\\\\\\\\\\\\"'):
                sub = sub.replace('\\\\\\\\\\\\"', '\\\\\\\\"')
            if '\\\\\\\\\\x' in sub:
                sub = sub.replace('\\\\\\\\\\x', '\\\\\\\\x')
            sub = ''
            i = 0
            while i < len(row)-1:
                if row[i] == '\\':
                    if row[i+1] == '\\':
                        sub += '\\\\\\\\'
                    elif row[i+1] == '"':
                        sub += '\\\\\\"'
                    elif row[i+1] == 'x':
                        sub += '\\\\x'
                    else:
                        print(row,sub,row[i],row[i+1])
                        raise
                    i += 2
                else:
                    sub += row[i]
                    i += 1
            sub += '"'
            #TODO: fix \\x 
            total += len(sub) + 4 - len(row)
        else:
            total += len(row) - len(eval(row))
    return total

output(day_eight(input), ts=1638334897)
output(day_eight(input, True), ts=1638335082)

1342                                                              [00:01:37]
2074                                                              [00:04:42]


# Day 9

In [4]:
input = get_input(9)

def day_nine(a, two=False):
    return a

output(day_nine(input), ts=1638334897)
output(day_nine(input, True), ts=1638335082)

1                                                                 [00:01:37]
2                                                                 [00:04:42]


# Day 10

In [4]:
input = get_input(10)

def day_ten(a, two=False):
    return a

output(day_ten(input), ts=1638334897)
output(day_ten(input, True), ts=1638335082)

1                                                                 [00:01:37]
2                                                                 [00:04:42]


# Day 11

In [4]:
input = get_input(11)

def day_eleven(a, two=False):
    return a

output(day_eleven(input), ts=1638334897)
output(day_eleven(input, True), ts=1638335082)

1                                                                 [00:01:37]
2                                                                 [00:04:42]


# Day 12

In [4]:
input = get_input(12)

def day_twelve(a, two=False):
    return a

output(day_twelve(input), ts=1638334897)
output(day_twelve(input, True), ts=1638335082)

1                                                                 [00:01:37]
2                                                                 [00:04:42]


# Day 13

In [4]:
input = get_input(13)

def day_thirteen(a, two=False):
    return a

output(day_thirteen(input), ts=1638334897)
output(day_thirteen(input, True), ts=1638335082)

1                                                                 [00:01:37]
2                                                                 [00:04:42]


# Day 14

In [4]:
input = get_input(14)

def day_fourteen(a, two=False):
    return a

output(day_fourteen(input), ts=1638334897)
output(day_fourteen(input, True), ts=1638335082)

1                                                                 [00:01:37]
2                                                                 [00:04:42]


# Day 15

In [4]:
input = get_input(15)

def day_fifteen(a, two=False):
    return a

output(day_fifteen(input), ts=1638334897)
output(day_fifteen(input, True), ts=1638335082)

1                                                                 [00:01:37]
2                                                                 [00:04:42]


# Day 16

In [4]:
input = get_input(16)

def day_sixteen(a, two=False):
    return a

output(day_sixteen(input), ts=1638334897)
output(day_sixteen(input, True), ts=1638335082)

1                                                                 [00:01:37]
2                                                                 [00:04:42]


# Day 17

In [4]:
input = get_input(17)

def day_seventeen(a, two=False):
    return a

output(day_seventeen(input), ts=1638334897)
output(day_seventeen(input, True), ts=1638335082)

1                                                                 [00:01:37]
2                                                                 [00:04:42]


# Day 18

In [4]:
input = get_input(18)

def day_eighteen(a, two=False):
    return a

output(day_eighteen(input), ts=1638334897)
output(day_eighteen(input, True), ts=1638335082)

1                                                                 [00:01:37]
2                                                                 [00:04:42]


# Day 19

In [4]:
input = get_input(19)

def day_nineteen(a, two=False):
    return a

output(day_nineteen(input), ts=1638334897)
output(day_nineteen(input, True), ts=1638335082)

1                                                                 [00:01:37]
2                                                                 [00:04:42]


# Day 20

In [4]:
input = get_input(20)

def day_twenty(a, two=False):
    return a

output(day_twenty(input), ts=1638334897)
output(day_twenty(input, True), ts=1638335082)

1                                                                 [00:01:37]
2                                                                 [00:04:42]


# Day 21

In [4]:
input = get_input(21)

def day_twentyone(a, two=False):
    return a

output(day_twentyone(input), ts=1638334897)
output(day_twentyone(input, True), ts=1638335082)

1                                                                 [00:01:37]
2                                                                 [00:04:42]


# Day 22

In [4]:
input = get_input(22)

def day_twentytwo(a, two=False):
    return a

output(day_twentytwo(input), ts=1638334897)
output(day_twentytwo(input, True), ts=1638335082)

1                                                                 [00:01:37]
2                                                                 [00:04:42]


# Day 23

In [4]:
input = get_input(23)

def day_twentythree(a, two=False):
    return a

output(day_twentythree(input), ts=1638334897)
output(day_twentythree(input, True), ts=1638335082)

1                                                                 [00:01:37]
2                                                                 [00:04:42]


# Day 24

In [4]:
input = get_input(24)

def day_twentyfour(a, two=False):
    return a

output(day_twentyfour(input), ts=1638334897)
output(day_twentyfour(input, True), ts=1638335082)

1                                                                 [00:01:37]
2                                                                 [00:04:42]


# Day 25

In [4]:
input = get_input(25)

def day_twentyfive(a):
    return a

output(day_twentyfive(input), ts=1638334897)

1                                                                 [00:01:37]
2                                                                 [00:04:42]
